# Setup

In [1]:
import os

import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory

from utils import (
    predict_batch, 
    parse_model_predicted_ratings, 
    parse_cot_output,
    sc_parsing_chain,
    cot_parsing_chain
)


In [2]:
load_dotenv("./.env")

True

In [13]:
os.environ['GOOGLE_API_KEY'] = os.environ['GOOGLE_GENAI_API_KEY']

# Data

In [4]:
DATA_DIR = "./data/"
reviews_path = os.path.join(DATA_DIR, "raw", "game_reviews", "reviews.csv")

reviews_df = pd.read_csv(reviews_path)

In [5]:
reviews_df.sample(5)

,app_id,app_name,review_text,review_score,review_votes,review_id
562062,570,Dota 2,"great game, very captivating :D",1,0,562062
331266,252490,Rust,Early Access Review,1,0,331266
454035,391540,Undertale,A game where you get infected and turned into ...,1,0,454035
554008,570,Dota 2,i love this game &lt;3,1,0,554008
166821,218620,PAYDAY 2,its a cool game the grapichs are real 9.5/10,1,0,166821


In [8]:
reviews_df.app_name.value_counts()

app_name
PAYDAY 2                         88973
Terraria                         84828
Rust                             77037
Dota 2                           73541
Rocket League                    54227
Undertale                        51918
Left 4 Dead 2                    50980
Warframe                         48229
Starbound                        41141
Portal 2                         38924
ARK: Survival Evolved            12470
Nether                            9595
Fallout 4                         6121
Godus                             4600
X Rebirth                         3370
Spacebase DF-9                    2790
Towns                             1766
Mafia III: Definitive Edition     1279
Name: count, dtype: int64

In [18]:
target_app = "Towns"
target_game_df = reviews_df.loc[reviews_df.app_name==target_app]
target_game_df = target_game_df.dropna()    

# Predicting Rating from Review Texts

In [11]:
PROMPT = """
ROLE:
-----
You are an expert Quality Controller in the Video Game Industry. 
You are proficient in analysing user reviews and quantifying the rating of a game based on the review.

TASK:
-----
You will be provided a user review, for that user review you will provide a rating from 1 - 9, based on the RATING SYSTEM.

RATING SYSTEM:
--------------
9 - Overwhelmingly Positive
8 - Very Positive
7 - Positive
6 - Mostly Positive
5 - Mixed
4 - Mostly Negative
3 - Negative
2 - Very Negative
1 - Overwhelmingly Negative

INPUT FORMAT:
-------------
Input is provided in the following format <REVIEW ID>|<REVIEW>.

OUTPUT:
-------
For each <REVIEW> present in the INPUT, 
provide the output as <REVIEW ID> (ID Of the Review from the Input) | <RATING> (Rating of the Review from the Rating system)

INPUT:
------
REVIEW ID|REVIEW
{}
"""

In [14]:
safety_settings={
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

llm = ChatGoogleGenerativeAI(model="gemini-pro", safety_settings=safety_settings)
llm_high_temperature = ChatGoogleGenerativeAI(model="gemini-pro", safety_settings=safety_settings, temperature=0.6)

llm.invoke("Say hello!")

AIMessage(content="Hello there! I'm happy to be here and ready to assist you with any questions or tasks you may have. Let me know how I can help!")

In [68]:
raw_predictions = predict_batch(model=llm, 
                                data=target_game_df,
                                prompt=PROMPT,
                                batch_size=20)
predicted_ratings = parse_model_predicted_ratings(raw_predictions)

Error while parsing predicted rating: Invalid rating: 216262 for rating string '216262|216262 | 6' for raw prediction 216262|216262 | 6
216263|216263 | 8
216264|216264 | 9
216265|216265 | 5
216266|216266 | 6
216267|216267 | 5
216268|216268 | 1
216269|216269 | 5
216270|216270 | 4
216271|216271 | 1
216272|216272 | 7
216273|216273 | 7
216274|216274 | 1
216275|216275 | 6
216276|216276 | 6
216277|216277 | 3
216278|216278 | 8
216279|216279 | 3
216280|216280 | 7
216281|216281 | 8
Error while parsing predicted rating: Invalid rating: 216263 for rating string '216263|216263 | 8' for raw prediction 216262|216262 | 6
216263|216263 | 8
216264|216264 | 9
216265|216265 | 5
216266|216266 | 6
216267|216267 | 5
216268|216268 | 1
216269|216269 | 5
216270|216270 | 4
216271|216271 | 1
216272|216272 | 7
216273|216273 | 7
216274|216274 | 1
216275|216275 | 6
216276|216276 | 6
216277|216277 | 3
216278|216278 | 8
216279|216279 | 3
216280|216280 | 7
216281|216281 | 8
Error while parsing predicted rating: Invali

In [72]:
predicted_ratings.rating.value_counts()

rating
1    710
2    206
3    166
4    132
6    129
7    127
5    125
8     99
9     46
Name: count, dtype: int64

In [74]:
# combining ratings into the original df
combined_zs_df = predicted_ratings.astype(str).merge(target_game_df.astype(str), on="review_id")
combined_zs_df.head()

,review_id,rating,app_id,app_name,review_text,review_score,review_votes,embeddings,cluster
0,214942,1,221020,Towns,TL;DR: DO NOT PURCHASE,-1,1,"[0.018348597, -0.02471556, -0.06385491, -0.041...",6
1,214943,1,221020,Towns,This highlights the worst part of Steam Greenl...,-1,1,"[0.0614244, -0.02122526, -0.07791973, -0.00037...",0
2,214944,1,221020,Towns,This game has been abandoned by the devs over ...,-1,1,"[0.06493835, -0.030919382, -0.03718793, -0.039...",5
3,214945,6,221020,Towns,I'd like to suggest an 'Abandoned' tag because...,-1,1,"[0.05155478, -0.028773155, -0.0977963, -0.0486...",4
4,214946,1,221020,Towns,Original Dev took the $2mil Alpha sales and sc...,-1,1,"[0.0510541, -0.017957242, -0.069919094, -0.024...",1


# Creating A Pipeline for COT & Self-Consistency

In [2]:
COT_PROMPT = """
ROLE:
-----
You are an expert Quality Controller in the Video Game Industry. 
You are proficient in analysing user reviews and quantifying the rating of a game based on the review.

TASK:
-----
You will be provided a user review, for that user review you will provide a rating from 1 - 9, based on the RATING SYSTEM.

RATING SYSTEM:
--------------
9 - Overwhelmingly Positive
8 - Very Positive
7 - Positive
6 - Mostly Positive
5 - Mixed
4 - Mostly Negative
3 - Negative
2 - Very Negative
1 - Overwhelmingly Negative

OUTPUT FORMAT:
--------------
The output should first provide a reasoning chain about what should be the rating enclosed within <reasoning_chain></reasoning_chain> tags.
At the last, the output should provide the final rating within the <rating></rating>

EXAMPLES:
---------

INPUT:
------

A game that strived to be as good as dwarf fortress from a developer who wasn't willing to put in the hours you need to be Toady.  An incomplete game to the point of not even being worth pirating.

OUTPUT:
-------

<reasoning_chain>
The game aimed to be like Dwarf Fortress but failed. The developer lacked dedication to put in the required effort.
The game is described as incomplete and unfinished. It is deemed not even worth pirating, implying no value.
There for the corresponding rating of the review should be 1.
</reasoning_chain>

<rating>
1
</rating>

INPUT:
------

A game that strived to be as good as dwarf fortress from a developer who wasn't willing to put in the hours you need to be Toady.  An incomplete game to the point of not even being worth pirating.

OUTPUT:
-------

<reasoning_chain>
The review indicates that the game suffers from a lack of proper guidance or tutorials, as the controls had to be 
looked up separately. Even after figuring out the controls, the gameplay remains confusing and unclear. Additionally, 
the reviewer found the experience quickly became boring and uninteresting, lacking any sense of intrigue or engagement. 
The confusing controls and lack of compelling gameplay elements result in an unsatisfying and tedious experience 
that the reviewer strongly advises against.
Therefore, the corresponding rating of the review should be 5.
.</reasoning_chain>

<rating>
2
</rating>

INPUT:
------

Personally, not my cup of tea. I suppose it's really just my 'taste', but I've played games that are similar to Towns, and had exceeded the quality of this game. I suppose if you want a 'bargain version' of this kind of game, then Towns would be the way to go.

OUTPUT:
------

<reasoning_chain>
The reviewer acknowledges that their negative opinion of the game "Towns" is largely a matter of personal taste. 
While they have played similar games that exceeded the quality of this one, they recognize that their critique 
is subjective. However, they suggest that for those seeking a more budget-friendly or "bargain" version of this 
type of game, "Towns" could potentially fit that niche. The reviewer seems to imply that while the game may not 
be exceptional, it could serve as an affordable option for those interested in the genre.
Therefore, the corresponding rating of the review should be 5.
.</reasoning_chain>

<rating>
5
</rating>

INPUT:
------

This is the strangely addictive love child of Majesty and Minecraft. It is extremely fun as it is, and has a tremendous potential for future improvement as well.

OUTPUT:
------

<reasoning_chain>
The review describes the game as an addictive combination of Majesty and Minecraft, implying a unique and 
engaging gameplay experience. It praises the game as being extremely fun in its current state. Additionally, 
it highlights the game's tremendous potential for further improvement in the future.
Therefore, the corresponding rating of the review should be 8.
.</reasoning_chain>

<rating>
8
</rating>

INPUT:
------

This is one of my favourite games on steam. I put hours into it and will continue to put hours into it. It has felt finished to me for a very long time, with the devs just continuing to add features. I honestly cannot bear to look at the forums since work on the game were discontinued, the attitudes of a lot of steam users who own this game are disgusting.  If you don't like this game, it isn't because it is 'unfinished' or 'falsely advertised', it is because this style of game isn't for you.  It was a great game when I bought it, it is a great game now.

OUTPUT:
------

<reasoning_chain>
The reviewer considers this one of their favorite games on Steam, having invested numerous hours into it.
They plan to continue playing for many more hours, indicating a high level of enjoyment and engagement.
Despite claims that the game is unfinished, the reviewer believes it has felt complete for a long time.
The developers have consistently added new features, further enhancing the experience. The reviewer dismisses 
negative attitudes, arguing that disliking the game is simply a matter of personal taste rather than objective flaws.
Therefore, the corresponding rating of the review should be 9.
</reasoning_chain>

<rating>
9
</rating>


INPUT:
------
{}

OUTPUT:
------
"""

In [104]:
# testing on a single record

row = target_game_df.iloc[3]
print(row.review_text, end="\n\n")
prediction = llm.invoke(COT_PROMPT.format(row.review_text))
print(prediction.content, end="\n\n")

I'd like to suggest an 'Abandoned' tag because of games like these

<reasoning_chain>
The review does not provide a clear opinion on the game's quality or experience. It merely suggests an "Abandoned" tag for games like it, implying neglect or lack of support. This does not provide sufficient information to determine a rating.
Therefore, the corresponding rating of the review should be 5.
</reasoning_chain>

<rating>
5
</rating>



In [125]:
cot_ratings = []
for idx, row in tqdm(target_game_df.iterrows()):
    cot_ratings.append(
        cot_parsing_chain(
            row.review_text, 
            COT_PROMPT, 
            llm, 
            parse_cot_output
        )
    )
cot_ratings_df = pd.DataFrame(cot_ratings)
cot_ratings_path = os.path.join(DATA_DIR, "processed", "game_reviews", "chain_of_thought_ratings.csv")
cot_ratings_df.to_csv(cot_ratings_path, index=False)

314it [18:42,  3.49s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The review is primarily a discussion about the game developer's personal situation and decisions, rather than a review of the game itself.
It provides no substantial feedback or evaluation of the game's gameplay, mechanics, or overall experience.
Therefore, it is not possible to determine a rating based on the provided information.
</reasoning_chain>

<rating>
N/A
</rating>


563it [33:19,  3.90s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The review simply states "Unfinished" without providing any further context or elaboration. This brief statement does not offer any specific insights into the game's quality, gameplay, or overall experience.
Therefore, it is not possible to determine a precise rating based solely on this limited information.
</reasoning_chain>

<rating>
N/A
</rating>


635it [37:39,  3.38s/it]

...

1686it [1:41:52,  3.33s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
No user review provided.
</reasoning_chain>

<rating>
N/A
</rating>


1687it [1:41:55,  3.16s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The review simply states that the game was made by Asians, which does not provide any meaningful insight into the quality or gameplay of the game.
Therefore, the corresponding rating of the review cannot be determined.
</reasoning_chain>

<rating>
N/A
</rating>


1765it [1:46:50,  3.63s/it]


In [176]:
sc_ratings = []
for idx, row in tqdm(target_game_df.iterrows()):
    final_rating, all_ratings = await sc_parsing_chain(
            row.review_text, 
            COT_PROMPT, 
            llm_high_temperature, 
            parse_cot_output
        )
    sc_ratings.append({"final": final_rating, "all": all_ratings, "review_id": row.review_id})
sc_ratings_df = pd.DataFrame(sc_ratings)
cot_ratings_path = os.path.join(DATA_DIR, "processed", "game_reviews", "self_consistency_ratings.csv")
sc_ratings_df.to_csv(cot_ratings_path, index=False)

3it [00:19,  6.94s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The reviewer's primary focus is on criticizing the game's developers and the moderators of its Steam forums, rather than providing an assessment of the game itself. The reviewer alleges that the developers have abandoned the game multiple times and that the forum moderators have been unfairly silencing negative feedback. The reviewer also provides evidence to support their claims, such as screenshots of forum posts and moderator actions. Therefore, it is difficult to determine a rating for the game based solely on this review.
</reasoning_chain>

<rating>
N/A
</rating>


4it [00:23,  5.59s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The reviewer's comment is not a review of a specific game, but rather a suggestion for a new tag to be added to the platform. The comment does not provide any specific feedback or evaluation of a particular game's quality, making it difficult to assign a rating.
Therefore, no rating can be assigned.
</reasoning_chain>

<rating>
N/A
</rating>


11it [00:57,  5.63s/it]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
21it [01:44,  4.54s/it]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
36it [02:58,  4.18s/it]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. c

...

1707it [2:52:43,  4.48s/it]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
1711it [2:53:07,  5.33s/it]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:158431097173'. [reason

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The reviewer's language is highly vulgar and explicitly sexual, making it challenging to provide a rating based solely on the content of the review. The review does not provide any meaningful insights into the game's gameplay, mechanics, or overall experience. The excessive use of profanities and sexual references obscures any potential valuable feedback or critique.

Therefore, it is not possible to determine an appropriate rating for this review.
</reasoning_chain>

<rating>
N/A
</rating>


1759it [2:59:26,  4.38s/it]

Error while parsing output: invalid literal for int() with base 10: '\nN/A\n'
Output:<reasoning_chain>
The review contains inappropriate and offensive language, including a death wish directed at the developer's wife.
It is not possible to provide a rating based on this review due to its lack of constructive feedback or relevant information about the game's quality.
</reasoning_chain>

<rating>
N/A
</rating>


1762it [2:59:40,  4.52s/it]Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
1765it [2:59:59,  6.12s/it]


# Analyzing the Results

In [ ]:
# removing the cols which doesn't match the review score
def does_review_score_match_rating(record, reverse=False):
    rating_to_review_score = -1 if int(record.predicted_rating) < 6 else 1
    result = rating_to_review_score == int(record.review_score) 
    return not result if reverse else result

In [ ]:
combined_sc_df = sc_ratings_df.astype(str).merge(target_game_df.astype(str), on="review_id")
combined_cot_df = cot_ratings_df.astype(str).merge(target_game_df.astype(str), on="review_id")

In [ ]:
(
    combined_zs_df
        .loc[combined_zs_df.predicted_rating != -1]
        .loc[combined_zs_df.apply(does_review_score_match_rating, axis=1)]
        .shape[0]
)

1429

In [189]:
(
    combined_cot_df
        .loc[combined_cot_df.predicted_rating != -1]
        .loc[combined_cot_df.apply(does_review_score_match_rating, axis=1)]
        .shape
)

(1622, 9)

In [194]:
(
    combined_sc_df
        .loc[combined_sc_df.final != -1]
        .loc[combined_sc_df.apply(does_review_score_match_rating, axis=1)]
        .shape
)

(1654, 10)

* zero_shot: 1429 / 1765
* cot: 1622 / 1765
* sc: 1654 / 1765

In [202]:
(
    combined_sc_df
        .loc[combined_sc_df.final != -1]
        .loc[:, "final"]
        .astype(int)
        .mean()
)

3.274787535410765

In [204]:
(
    combined_cot_df
        .loc[combined_cot_df.predicted_rating != -1]
        .loc[:, "predicted_rating"]
        .astype(int)
        .mean()
)

3.2903593839132914

In [205]:
(
    combined_zs_df
        .loc[combined_zs_df.rating != -1]
        .loc[:, "rating"]
        .astype(int)
        .mean()
)

3.2200128287363694